In [ ]:
# Import Libraries
import numpy as np
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt

Part 1: Look through the code and understand how each of the functions work, and what each of their inputs and outputs mean. Take particular care to understand what happens with the function arguments that are called inside the exp_max function, and the dimensions of the matrices involved. Add appropriate comments to your copy of the code. 

In [ ]:
def exp_max(Iter, K, pdf, train, Xmat, W_Init, P_Init):
    """
    Implements the Expectation-Maximization (EM) algorithm for clustering.

    Args:
        Iter (int): The number of iterations for which the algorithm should run.
        K (int): The number of clusters.
        pdf (function): A probability density function (e.g., normal_pdf) that takes parameters
                        (e.g., mean) and data (Xmat) to return probabilities.
        train (function): A learning function (e.g., normal_train) that fits the optimal parameters
                          (e.g., mean) based on cluster probabilities and data.
        Xmat (numpy.ndarray): An array storing the training data. Dimensions: (n_samples, n_features).
        W_Init (numpy.ndarray): The initial weights for each cluster. Dimensions: (1, K).
        P_Init (numpy.ndarray): The initial choice of parameters for each cluster (e.g., initial means).
                                Dimensions: (n_features, K).

    Returns:
        tuple: A tuple containing:
            - W (numpy.ndarray): The final weights for each cluster. Dimensions: (1, K).
            - P (numpy.ndarray): The final parameters for each cluster (e.g., final means).
                                 Dimensions: (n_features, K).
            - p (numpy.ndarray): The final probabilities associated with every cluster and data point.
                                 Dimensions: (K, n_samples).
    """
    n, D = Xmat.shape  # n = number of samples, D = number of features
    p = np.zeros((K, n))  # Initialize probabilities: K clusters x n samples
    W, P = W_Init, P_Init  # Initialize weights and parameters

    for i in range(0, Iter):
        # E-Step (Expectation Step): Calculate the probability of each data point belonging to each cluster
        for k in range(0, K):
            # p[k,:] calculates the likelihood of each data point Xmat under the current parameters P[:,k]
            # and multiplies by the current weight W[0,k] for cluster k.
            # pdf(P[:,k], Xmat) returns a 1D array of probabilities for each sample.
            p[k,:] = W[0,k] * pdf(P[:,k], Xmat)

        # Normalize probabilities so that for each data point, the sum of probabilities across all clusters is 1.
        # p becomes the responsibility matrix (gamma in EM literature).
        p = (p / np.sum(p, axis=0)) # Sum along axis 0 (columns) to normalize for each data point

        # M-Step (Maximization Step): Update weights and parameters based on the calculated responsibilities
        # Update weights (W): The new weight for each cluster is the average responsibility assigned to it.
        W = np.mean(p, axis=1).reshape(1, K) # Mean along axis 1 (rows) to get average responsibility for each cluster

        # Update parameters (P): For each cluster, train a new set of parameters using the weighted data points.
        # The 'train' function takes the responsibilities for a cluster (p[k,:]) and the data (Xmat).
        for k in range(0, K):
            P[:,k] = train(p[k,:], Xmat)
    return W, P, p

def normal_train(p, Xmat):
    """
    Trains the mean parameter for a Gaussian distribution.

    Args:
        p (numpy.ndarray): The responsibilities (probabilities) of data points belonging to a specific cluster.
                           Dimensions: (n_samples,).
        Xmat (numpy.ndarray): The training data. Dimensions: (n_samples, n_features).

    Returns:
        numpy.ndarray: The calculated mean vector for the cluster. Dimensions: (n_features,).
    """
    # Calculate the weighted mean of the data points based on their responsibilities.
    # (Xmat.T @ p.T) performs a weighted sum of data points for each feature.
    # sum(p) is the sum of responsibilities for the cluster.
    m = (Xmat.T @ p.T) / np.sum(p)
    return m

def normal_pdf(m, Xmat, var=1):
    """
    Calculates the probability density function (PDF) for a multivariate normal distribution.
    Assumes a fixed, symmetric variance.

    Args:
        m (numpy.ndarray): The mean vector of the Gaussian distribution. Dimensions: (n_features,).
        Xmat (numpy.ndarray): The data points for which to calculate the PDF. Dimensions: (n_samples, n_features).
        var (float): The fixed variance for the symmetric Gaussian. Defaults to 1.

    Returns:
        numpy.ndarray: An array of PDF values for each data point. Dimensions: (n_samples,).
    """
    C = np.eye(Xmat.shape[1]) * var # Create a diagonal covariance matrix with 'var' on the diagonal
    mvn = multivariate_normal(mean=m.T, cov=C)
    return mvn.pdf(Xmat)


# --- Q5: K-Means Algorithm (with comments) ---

def kmeans(eps, K, Xmat, c_init):
    """
    Implements the K-Means clustering algorithm.

    Args:
        eps (float): The convergence threshold. The algorithm stops when the change in centroids
                     is less than this value.
        K (int): The number of clusters.
        Xmat (numpy.ndarray): The data points to be clustered. Dimensions: (n_samples, n_features).
        c_init (numpy.ndarray): The initial centroids for each cluster. Dimensions: (n_features, K).

    Returns:
        tuple: A tuple containing:
            - c (numpy.ndarray): The final centroids for each cluster. Dimensions: (n_features, K).
            - label (numpy.ndarray): An array indicating the cluster assignment for each data point.
                                   Dimensions: (n_samples,).
    """
    n, D = Xmat.shape  # n = number of samples, D = number of features
    c = c_init.copy()  # Initialize centroids
    c_old = np.zeros(c.shape)  # To store old centroids for convergence check
    dist2 = np.zeros((K, n))  # To store squared distances from each point to each centroid

    # Iterate until centroids converge
    while np.abs(c - c_old).sum() > eps:
        c_old = c.copy()

        # E-Step (Assignment Step): Assign each data point to the nearest centroid
        for i in range(0, K):  # Compute the squared distances from each point to centroid i
            # (Xmat - c[:,i].T) calculates the difference between each data point and the current centroid i.
            # **2 squares these differences.
            # np.sum(..., 1) sums the squared differences across features to get the squared Euclidean distance.
            dist2[i,:] = np.sum((Xmat - c[:,i].T)**2, 1)

        # Assign each point to the cluster whose centroid is closest
        label = np.argmin(dist2, 0)  # label[j] is the index of the closest centroid for data point j

        # M-Step (Update Step): Recompute the centroids based on the new assignments
        for i in range(0, K):  # For each cluster
            entries = np.where(label == i)  # Get indices of data points assigned to cluster i
            if len(entries[0]) > 0: # Ensure there are points in the cluster to avoid division by zero
                # Recompute centroid i as the mean of all data points assigned to it
                # entries[0] contains the indices, so we use Xmat[entries[0],:] to get the points
                c[:,i] = np.mean(Xmat[entries[0],:], axis=0)
            else:
                # If a cluster becomes empty, keep its centroid in place or reinitialize (here, keeping in place)
                pass
    return c, label